In [1]:
from carbon.helpers.stdimports import *

# those are not available at the helpers level
from carbon.helpers.version import VersionRequirementNotMetError
from carbon.helpers.simulation import run_sim, plot_sim
from carbon.helpers.widgets import CheckboxManager, DropdownManager, PcSliderManager

# that's part of the test, that all those import from the helpers level
from carbon.helpers import fload, fsave
from carbon.helpers import Params
from carbon.helpers import dfread, pdread, pathtime, pathtime_yrs, pathtime_days, pdcols, j
from carbon.helpers import strategy
from carbon.helpers import require_version
from carbon.helpers import print_version

plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
print_version(require="2.3")

[stdimports] imported np, pd, plt, os, sqrt, exp, log
Carbon v2.3-BETA0 (23/Jan/2022)
CarbonSimulatorUI v2.4 (12/Jan/2023)
CarbonOrderUI v1.6.1 (21/Jan/2023)


# Carbon Helpers (NBTest 052)

## params

In [2]:
p = Params(a=1, b=2)
assert p["a"] == 1
assert p.a == 1
assert p["c"] is None
print(p)
assert str(p) == "Params.construct({'a': 1, 'b': 2})"
assert p.params == {'a': 1, 'b': 2}
p["c"] = 5
assert p["c"] == 5
assert p.c == 5
result = p.add(d=10, e=11)
assert result is p
assert p["d"] == 10
assert p.e == 11
try:
    p.z
    raise RuntimeError("should raise")
except KeyError as e:
    print(e)

Params.construct({'a': 1, 'b': 2})
'z'


In [3]:
p = Params(a=1, b=2)
assert p.defaults == {}
p.set_default(b=20, c=3)
assert str(p) == "Params.construct({'a': 1, 'b': 2}, defaults={'b': 20, 'c': 3})"
assert p.b == 2
assert p.get_default("b") == 20
assert p.c == 3

p = Params(a=1, b=2)
assert p.get_default("c") is None
result = p.set_default()
assert result is p
p.set_default(c=10, d=11)
assert p.c == 10
assert p.d == 11
assert p.defaults == {'c': 10, 'd': 11}
assert p["e"] is None
try:
    p.e
    raise RuntimeError("should raise")
except KeyError as e:
    pass

In [4]:
p = Params.construct({'a': 1, 'b': 2}, defaults={'b': 20, 'c': 3})
assert p.a == 1
assert p.b == 2
assert p.c == 3
assert p.get_default("b") == 20
pp = Params.construct(p)
assert not pp is p
assert p.params == pp.params
try:
    ppp = Params.construct(p, defaults={"e":100})
except ValueError as e:
    print(e)

('Must not provide default in both constructor and dct', Params.construct({'a': 1, 'b': 2}, defaults={'b': 20, 'c': 3}), {'e': 100})


In [5]:
p  = Params.construct({'a': 1, 'b': 2})
defaults = {'b': 20, 'c': 3}
pp = Params.construct(p, defaults=defaults)
assert not pp is p
assert p.params == pp.params
assert pp.defaults == defaults
assert not pp.defaults is defaults
assert not pp._defaults is defaults

In [6]:
p = Params.construct()
assert p.params == {}
p = Params.construct(None)
assert p.params == {}

## widgets [NOTEST]

https://ipywidgets.readthedocs.io/en/7.6.3/examples/Widget%20Styling.html#The-Flexbox-layout

In [7]:
CM1 = CheckboxManager([i for i in range(10)])
CM1(vertical=False)

In [8]:
CM2 = CheckboxManager({i:678975437*(i+10) for i in range(5)}, objects=("a", "b", "c", "d", "e"))
CM2()

In [9]:
CM2.objects(), CM2.objects(checked=False)

((), ('a', 'b', 'c', 'd', 'e'))

In [10]:
SM = PcSliderManager([f"Slider #{i+1}" for i in range(5)])
SM()

In [11]:
SM.values

(0.0, 0.0, 0.0, 0.0, 0.0)

In [12]:
DD = DropdownManager([678975437*(i+10) for i in range(5)])
DD()

Dropdown(options=(('6789754370', '6789754370'), ('7468729807', '7468729807'), ('8147705244', '8147705244'), ('…

In [13]:
print(CM1.values, CM2.values, DD.value)

(False, False, False, False, False, False, False, False, False, False) (False, False, False, False, False) 6789754370


## widgets

In [14]:
CM = CheckboxManager([1,2,3], None)
assert CM.choice_ids == ('1', '2', '3')
assert CM.choice_descr == ('1', '2', '3')
assert CM.values == (False, False, False)
assert CheckboxManager({1:11, 2:12}).choice_ids == ('1', '2')
assert CheckboxManager({1:11, 2:12}).choice_descr == ('11', '12')
assert CheckboxManager([1,2,3], False).values == (False, False, False)
assert CheckboxManager([1,2,3], True).values == (True, True, True)
assert CheckboxManager([1,2,3], [10,0,1]).values == (True, False, True)
assert CheckboxManager([1,2,3], disabled=[10,0,1]).disabled == (True, False, True)
assert CheckboxManager.from_idvdct({10:10,12:0}).choice_ids == ('10', '12')
assert CheckboxManager.from_idvdct({10:10,12:0}).choice_descr == ('10', '12')
assert CheckboxManager.from_idvdct({10:10,12:0}).values == (True, False)
try:
    CheckboxManager([1,2,3], [10,0])
    raise
except ValueError as e:
    print(e)
try:
    CheckboxManager([1,2,3], disabled=[10,0])
    raise
except ValueError as e:
    print(e)

('len(values) [2] must be len(choices) [3]', [10, 0], [1, 2, 3])
('len(disabled) [2] must be len(choices) [3]', [10, 0], [1, 2, 3])


In [15]:
assert PcSliderManager({1:11, 2:12}).choice_ids == ('1', '2')
assert PcSliderManager({1:11, 2:12}).choice_descr == ('11', '12')
assert PcSliderManager([1,2,3], None).values == (0.,)*3
assert PcSliderManager([1,2,3], 0.3).values == (0.3, 0.3, 0.3)
assert PcSliderManager([1,2,3], [2,0.5,-1]).values == (1, 0.5, 0)
assert PcSliderManager([1,2,3], disabled=[10,0,1]).disabled == (True, False, True)
try:
    PcSliderManager([1,2,3], [10,0])
    raise
except ValueError as e:
    print(e)
try:
    PcSliderManager([1,2,3], disabled=[10,0])
    raise
except ValueError as e:
    print(e)

('len(values) [2] must be len(choices) [3]', [10, 0], [1, 2, 3])
('len(disabled) [2] must be len(choices) [3]', [10, 0], [1, 2, 3])


In [16]:
assert DropdownManager([1,2,3]).options == ('1', '2', '3')
assert DropdownManager([1,2,3]).labels == ('1', '2', '3')
assert DropdownManager([1,2,3], labels=[10,20,30]).options == ('1', '2', '3')
assert DropdownManager([1,2,3], labels=[10,20,30]).labels == ('10', '20', '30')
assert DropdownManager({1:10, 2:20, 3:30}).options == ('1', '2', '3')
assert DropdownManager({1:10, 2:20, 3:30}).labels == ('10', '20', '30')
assert DropdownManager({1:10, 2:20, 3:30}).value == "1"
assert DropdownManager({1:10, 2:20, 3:30}).label == "10"
assert DropdownManager({1:10, 2:20, 3:30}).result == ("1", "10")

## helpers stdimport

check that the following objects have been imported

In [17]:
np
plt
pd
CarbonSimulatorUI
CarbonOrderUI
P

carbon.pair.CarbonPair

In [18]:
sqrt
exp
log

<function math.log>

## helpers version

In [19]:
assert require_version("1.0", "1.0", raiseonfail=False) == True
assert require_version("2.0", "1.0", raiseonfail=False) == False
assert require_version("1.0", "2.0", raiseonfail=False) == True
assert require_version("1.0.1", "1.0", raiseonfail=False) == False
assert require_version("1.0", "1.0.1", raiseonfail=False) == True
assert require_version("1.0", "11.0", raiseonfail=False) == True
assert require_version("11.0", "1.0", raiseonfail=False) == False
assert require_version("1.3beta1", "1.3", raiseonfail=False) == True
assert require_version("1.3beta1", "1.3beta1", raiseonfail=False) == True
assert require_version("1.3-1", "1.3-2", raiseonfail=False) == True
assert require_version("1.3-2", "1.3-1", raiseonfail=False) == True
assert require_version("1.0", "1.0", raiseonfail=False) == True
assert require_version("1.0", "1.0", raiseonfail=False) == True

obviously must be tested with version >= 2.0

In [20]:
require_version("2.0")

In [21]:
require_version("1.0", "1.0")

In [22]:
try:
    require_version("2.0", "1.0")
    run("must raise error")
except VersionRequirementNotMetError as e:
    print(e)

('Version requirements not met (required = [2, 0], actual = [1, 0])', '2.0', '1.0')


## helpers strategy

In [23]:
# from carbon.helpers import strategy as _strategy
# help(_strategy)

In [24]:
assert strategy.from_mgw(m=100) == strategy.from_mgw()
assert strategy.from_mgw(g=0) == strategy.from_mgw()
assert strategy.from_mgw(w=0) == strategy.from_mgw()

In [25]:
assert strategy.from_mgw(g=0.1).p_buy_a == 100/(1.05)
assert strategy.from_mgw(g=0.1).p_sell_a == 100*(1.05)

In [26]:
assert strategy.from_mgw(g=0.1, w=0.1).p_buy_b == 100/1.05/1.1
assert strategy.from_mgw(g=0.1, w=0.1).p_sell_b == 100*1.05*1.1

In [27]:
assert strategy.from_mgw().slashpair == "RSK/CSH"
assert strategy.from_mgw(rsk="ETH", csh="USD").slashpair == "ETH/USD"

## helpers pdread

In [28]:
# from carbon.helpers import pdread as _pdread
# help(_pdread)

In [29]:
DATAFN = "resources/data/RAN-050-00.pickle"
#DATAFN = "../data/RAN-050-00.pickle"

In [30]:
assert len(pdread(DATAFN))==101
assert pdread(DATAFN).iloc[0].index[5] == "p0005"
assert str(pdread(DATAFN).index[0]) == '2020-01-01 00:00:00'
assert len(pdread(DATAFN)["p0000"]) == len(pdread(DATAFN))
assert list(pdread(DATAFN)["p0000"]) == list(pdread(DATAFN, "p0000"))

In [31]:
# those assertions should be (mostly) independent from the test data used
ONEDAY = pd.Timedelta(days=1)
FDAY, LDAY = pdread(DATAFN).index[0], pdread(DATAFN).index[-1]
NDAY = LDAY - FDAY
assert pdread(DATAFN, from_ts=FDAY - 100*ONEDAY).index[-1] == LDAY
assert pdread(DATAFN, from_ts=LDAY - 100*ONEDAY).index[-1] == LDAY
assert pdread(DATAFN, from_ts=FDAY + 100*ONEDAY).index[-1] == LDAY
assert len(pdread(DATAFN, from_ts=LDAY + 1*ONEDAY))==0
assert pdread(DATAFN, from_ts=FDAY+20*ONEDAY).index[0] >= FDAY
assert pdread(DATAFN, period_days=NDAY/ONEDAY+10).index[-1] == LDAY
assert pdread(DATAFN, period_days=NDAY/ONEDAY-10).index[-1] <= LDAY
assert len(pdread(DATAFN, period_days=0))==1

In [32]:
# those assertions depend on the exact test data used
assert str(pdread(DATAFN                ).index[-1]) == '2020-12-31 06:00:00'
assert str(pdread(DATAFN, period_days=20).index[-1]) == '2020-01-19 06:18:00'
assert str(pdread(DATAFN, period_days=50).index[-1]) == '2020-02-17 11:34:48'

In [33]:
# those assertions depend on the exact test data used
assert str(pdread(DATAFN, from_ts="2000-01-01").index[0])  == '2020-01-01 00:00:00'
assert str(pdread(DATAFN, from_ts="2020-01-02").index[0])  == '2020-01-04 15:39:36'
assert str(pdread(DATAFN, from_ts="2020-01-02").index[-1]) == '2020-12-31 06:00:00'
assert str(pdread(DATAFN, from_ts="2020-05-02").index[-1]) == '2020-12-31 06:00:00'

In [34]:
# those assertions depend on the exact test data used
assert str(pdread(DATAFN, from_ts="2000-01-01", period_days=30).index[0])   == '2020-01-01 00:00:00'
assert str(pdread(DATAFN, from_ts="2000-01-01", period_days=30).index[-1])  == '2020-01-30 05:16:48'
assert str(pdread(DATAFN, from_ts="2020-03-01", period_days=30).index[0])   == '2020-03-03 02:13:12'
assert str(pdread(DATAFN, from_ts="2020-03-01", period_days=30).index[-1])  == '2020-03-28 15:50:24'

In [35]:
assert pathtime(pdread(DATAFN, "p0000")) == 1       # DEPRECATED
assert pathtime_yrs(pdread(DATAFN, "p0000")) == 1
assert pathtime_days(pdread(DATAFN, "p0000")) == 365.25

In [36]:
assert len(pdcols(DATAFN)) == 1000
assert (pdcols(DATAFN) == pdread(DATAFN).columns).all() == True
assert len(pdread(DATAFN, period_days=10)) == 3
assert len(pdread(DATAFN, period_pc=0.1)) == 11
assert len(pdread(DATAFN, from_ts="2020-07-01")) == 51
assert len(pdread(DATAFN, from_pc=0.5)) == 51